In [1]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()


In [8]:
import numpy as np
from PIL import Image

def update(change):
    image_array = change.get('new')
    if image_array is None or not isinstance(image_array, np.ndarray):
        print("Error: 'new' 必須是一個 NumPy 陣列。")
        return

    try:
        # 確保數據範圍
        if image_array.max() > 1.0:
            image_array = image_array / 255.0
        image_array = np.clip(image_array, 0, 1)

        # 檢查並修正顏色通道順序 (BGR -> RGB)
        if image_array.shape[2] == 3:  # 確保有三個通道
            image_array = image_array[:, :, ::-1]  # 反轉通道順序

        # 將 NumPy 陣列轉換為 PIL Image
        image = Image.fromarray((image_array * 255).astype(np.uint8))
        image.save("now.jpg", format="JPEG")
        with open("now.jpg", "rb") as image_file:
            # Send the POST request with the file
            response = requests.post(
                "https://openai.chinococo.tw/analyze-image",
                files={"file": image_file}  # Key "file" is commonly expected by servers
            )
        response_content = response.content.decode('utf-8')  # Decode the raw bytes
        try:
            # Parse JSON response
            response_json = json.loads(response_content)
            # Extract and print the "result" field
            result_text = response_json.get("result", "No result found")
            print("Response status code:", response.status_code)
            print("Decoded response content:", result_text)
        except json.JSONDecodeError:
            print("Failed to parse JSON response")
            print("Raw response content:", response_content)

    except Exception as e:
        print(f"Error: 無法將 NumPy 陣列轉換為圖片 - {e}")

# 測試例子
example_array = np.random.rand(224, 224, 3)  # 模擬一個隨機 RGB 圖片
update({"new": example_array})


Response status code: 200
Decoded response content: 抱歉，我無法分析或識別這張圖片的內容。


In [ ]:
while(1):
    import time
    # 測試執行速度
    update({'new': camera.value})
    time.sleep(10)


Response status code: 200
Decoded response content: 抱歉，我無法分析圖片內容或路牌標示。你可以描述圖片，我會根據描述提供幫助。
Response status code: 200
Decoded response content: 抱歉，我無法分析或描述圖片中的內容。若有其他問題或需要的幫助，請告訴我！
Response status code: 200
Decoded response content: 抱歉，我無法分析這張圖片的內容。
Response status code: 200
Decoded response content: 圖片中顯示的路牌是一個警告標誌，標誌上有一個行人，顯示這是一個有行人穿越的地方的提示。這通常用於提醒駕駛者注意行人，以確保行人的安全。該標誌通常呈現三角形，背景為白色，邊框和行人圖案為黑色或紅色。


In [ ]:
camera.stop()